# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import optuna
from scipy import stats


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Load data

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"
path_configs = "E:/Git/bbs/notebooks/mriya/configs"

feats_set = 'Sphygmocardiography'

path_ckpts = f"E:/Git/bbs/notebooks/mriya/pt/{feats_set}"

tst_n_splits = 5
tst_n_repeats = 5
tst_random_state = 1337
tst_split_id = 24

val_n_splits = 4
val_n_repeats = 4
val_random_state = 1337
val_fold_id = 2

data = pd.read_excel(f"{path}/{feats_set}/data.xlsx", index_col=0)
# df_feats = pd.read_excel(f"{path}/{feats_set}/feats.xlsx", index_col=0)
df_feats = pd.read_excel(f"{path}/{feats_set}/feats_cont.xlsx", index_col=0)

feat_trgt = 'Age'
feats_cnt = df_feats.index[df_feats['data_type'].isin(['decimal', 'integer'])].to_list()
feats_cat = df_feats.index[df_feats['data_type'].isin(['enum'])].to_list()

feats = list(feats_cnt) + list(feats_cat)

In [ ]:
df_corr = pd.DataFrame(index=feats_cnt, columns=['rho'])
for f in tqdm(feats_cnt):
    df_tmp = data.loc[:, [feat_trgt, f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        vals_1 = df_tmp.loc[:, feat_trgt].values
        vals_2 = df_tmp.loc[:, f].values
        df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
df_corr.dropna(axis=0, how='any', inplace=True)
df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
feats_cnt_wo_age = df_corr.index.to_list()
df_corr = df_corr.apply(pd.to_numeric)

sns.set_theme(style='ticks')

fig, ax = plt.subplots(figsize=(0.4 + 0.04 * df_corr.index.str.len().max() + 0.1 * len(feats_cnt_wo_age), 0.5 + 0.4 * len(feats_cnt_wo_age) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
# fig, ax = plt.subplots(layout='constrained')
heatmap = sns.heatmap(
    df_corr.loc[:, ['rho']],
    annot=True,
    fmt=".2f",
    vmin=-1.0,
    vmax=1.0,
    cmap='coolwarm',
    linewidth=0.1,
    linecolor='black',
    #annot_kws={"fontsize": 15},
    cbar_kws={
        # "shrink": 0.9,
        # "aspect": 30,
        #'fraction': 0.046, 
        #'pad': 0.04,
    },
    ax=ax
)
heatmap_pos = ax.get_position()
ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('')
ax.set_ylabel('')
ax.set(xticklabels=[])
ax.set(xticks=[])
plt.savefig(f"{path}/{feats_set}/age_feats_pearsonr.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/{feats_set}/age_feats_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

# Load stratification

In [ ]:
with open(f"{path}/{feats_set}/samples_tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_val({val_random_state}_{val_n_splits}_{val_n_repeats}).pickle", 'rb') as handle:
    samples = pickle.load(handle)

for split_id in range(tst_n_splits * tst_n_repeats):
    for fold_id in range(val_n_splits * val_n_repeats):
        test_samples = samples[split_id]['test']
        train_samples = samples[split_id]['trains'][fold_id]
        validation_samples = samples[split_id]['validations'][fold_id]

        intxns = {
            'train_validation': set.intersection(set(train_samples), set(validation_samples)),
            'validation_test': set.intersection(set(validation_samples), set(test_samples)),
            'train_test': set.intersection(set(train_samples), set(test_samples))
        }
        
        for intxn_name, intxn_samples in intxns.items():
            if len(intxn_samples) > 0:
                print(f"Non-zero {intxn_name} intersection ({len(intxn_samples)}) for {split_id} Split and {fold_id} Fold!")

# Train, Validation, Test selection

In [ ]:
split_dict = samples[tst_split_id]

test = data.loc[split_dict['test'], feats + [feat_trgt]]
train = data.loc[split_dict['trains'][val_fold_id], feats + [feat_trgt]]
validation = data.loc[split_dict['validations'][val_fold_id], feats + [feat_trgt]]

# Optuna training

## Models setup

In [ ]:
seed_target = 451  # 1337 42 451 1984 1899 1408

models_runs = {
    # 'GANDALF': {
    #     'config': GANDALFConfig,
    #     'n_trials': 1024,
    #     'seed': seed_target,
    #     'n_startup_trials': 256,
    #     'n_ei_candidates': 16
    # },
    # 'FTTransformer': {
    #     'config': FTTransformerConfig,
    #     'n_trials': 1024,
    #     'seed': seed_target,
    #     'n_startup_trials': 256,
    #     'n_ei_candidates': 16
    # },
    'DANet': {
        'config': DANetConfig,
        'n_trials': 512,
        'seed': seed_target,
        'n_startup_trials': 256,
        'n_ei_candidates': 16
    },
    # 'CategoryEmbeddingModel': {
    #     'config': CategoryEmbeddingModelConfig,
    #     'n_trials': 256,
    #     'seed': seed_target,
    #     'n_startup_trials': 64,
    #     'n_ei_candidates': 16
    # },
    # 'TabNetModel': {
    #     'config': TabNetModelConfig,
    #     'n_trials': 256,
    #     'seed': seed_target,
    #     'n_startup_trials': 64,
    #     'n_ei_candidates': 16
    # }
}

## Training

In [ ]:
%%capture

dfs_models = []

for model_name, model_run in models_runs.items():

    model_config_name = model_run['config']
    n_trials = model_run['n_trials']
    seed = model_run['seed']
    n_startup_trials = model_run['n_startup_trials']
    n_ei_candidates = model_run['n_ei_candidates']

    data_config = read_parse_config(f"{path_configs}/DataConfig.yaml", DataConfig)
    data_config['target'] = [feat_trgt]
    data_config['continuous_cols'] = feats_cnt
    data_config['categorical_cols'] = feats_cat
    trainer_config = read_parse_config(f"{path_configs}/TrainerConfig.yaml", TrainerConfig)
    pathlib.Path(path_ckpts).mkdir(parents=True, exist_ok=True)
    trainer_config['checkpoints_path'] = path_ckpts
    optimizer_config = read_parse_config(f"{path_configs}/OptimizerConfig.yaml", OptimizerConfig)

    lr_find_min_lr = 1e-8
    lr_find_max_lr = 10
    lr_find_num_training = 256
    lr_find_mode = "exponential"
    lr_find_early_stop_threshold = 8.0

    trainer_config['seed'] = seed
    trainer_config['checkpoints'] = 'valid_loss'
    trainer_config['load_best'] = True
    trainer_config['auto_lr_find'] = False

    model_config_default = read_parse_config(f"{path_configs}/models/{model_name}Config.yaml", model_config_name)
    tabular_model_default = TabularModel(
        data_config=data_config,
        model_config=model_config_default,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=False,
    )
    datamodule = tabular_model_default.prepare_dataloader(train=train, validation=validation, seed=seed)

    opt_parts = ['test', 'validation']
    opt_metrics = [('mean_absolute_error', 'minimize')]
    opt_directions = []
    for part in opt_parts:
        for metric_pair in opt_metrics:
            opt_directions.append(f"{metric_pair[1]}")

    trials_results = []

    study = optuna.create_study(
        study_name=model_name,
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=n_startup_trials,
            n_ei_candidates=n_ei_candidates,
            seed=seed,
        ),
        directions=opt_directions
    )
    study.optimize(
        func=lambda trial: train_hyper_opt(
            trial=trial,
            trials_results=trials_results,
            opt_metrics=opt_metrics,
            opt_parts=opt_parts,
            model_config_default=model_config_default,
            data_config_default=data_config,
            optimizer_config_default=optimizer_config,
            trainer_config_default=trainer_config,
            experiment_config_default=None,
            train=train,
            validation=validation,
            test=test,
            datamodule=datamodule,
            min_lr=lr_find_min_lr,
            max_lr=lr_find_max_lr,
            num_training=lr_find_num_training,
            mode=lr_find_mode,
            early_stop_threshold=lr_find_early_stop_threshold
        ),
        n_trials=n_trials,
        show_progress_bar=True
    )

    fn_trials = (
        f"model({model_name})_"
        f"trials({n_trials}_{seed}_{n_startup_trials}_{n_ei_candidates})_"
        f"tst({tst_split_id})_"
        f"val({val_fold_id})"
    )

    df_trials = pd.DataFrame(trials_results)
    df_trials['split_id'] = tst_split_id
    df_trials['fold_id'] = val_fold_id
    df_trials["train_more"] = False
    df_trials.loc[(df_trials["train_loss"] > df_trials["test_loss"]) | (
            df_trials["train_loss"] > df_trials["validation_loss"]), "train_more"] = True
    df_trials["validation_test_mean_loss"] = (df_trials["validation_loss"] + df_trials["test_loss"]) / 2.0
    df_trials["train_validation_test_mean_loss"] = (df_trials["train_loss"] + df_trials["validation_loss"] + df_trials["test_loss"]) / 3.0
    df_trials.sort_values(by=['test_loss'], ascending=[True], inplace=True)
    df_trials.style.background_gradient(
        subset=[
            "train_loss",
            "validation_loss",
            "validation_test_mean_loss",
            "train_validation_test_mean_loss",
            "test_loss",
            "time_taken",
            "time_taken_per_epoch"
        ], cmap="RdYlGn_r"
    ).to_excel(f"{trainer_config['checkpoints_path']}/{fn_trials}.xlsx")

    dfs_models.append(df_trials)

df_models = pd.concat(dfs_models, ignore_index=True)
df_models.insert(0, 'Selected', 0)
fn = (
    f"models_"
    f"tst({tst_split_id})_"
    f"val({val_fold_id})"
)
df_models.style.background_gradient(
    subset=[
        "train_loss",
        "validation_loss",
        "validation_test_mean_loss",
        "train_validation_test_mean_loss",
        "test_loss",
        "time_taken",
        "time_taken_per_epoch"
    ], cmap="RdYlGn_r"
).to_excel(f"{trainer_config['checkpoints_path']}/{fn}.xlsx")

# Best models analysis

In [ ]:
from regression_bias_corrector import LinearBiasCorrector

is_explain = True
explain_method = "GradientShap"
explain_baselines = "b|1000"
explain_n_feats_to_plot = 25

models_type = 'DANet'
models_ids = [
438
]
models_ids = sorted(list(set(models_ids)))

path_to_models = path_ckpts

df_sweeps = pd.read_excel(f"{path_to_models}/progress.xlsx", index_col=0)

path_to_candidates = f"{path_to_models}/candidates/{models_type}"
pathlib.Path(path_to_candidates).mkdir(parents=True, exist_ok=True)
df_sweeps.loc[models_ids, :].style.background_gradient(
    subset=[
        "train_loss",
        "validation_loss",
        "test_loss",
        "time_taken",
        "time_taken_per_epoch"
    ], cmap="RdYlGn_r"
).to_excel(f"{path_to_candidates}/selected.xlsx")

for model_id in models_ids:

    model_dir = str(pathlib.Path(df_sweeps.at[model_id, 'checkpoint']).parent).replace('\\', '/') + '/' + pathlib.Path(df_sweeps.at[model_id, 'checkpoint']).stem
    model = TabularModel.load_model(model_dir)
    pathlib.Path(f"{path_to_candidates}/{model_id}").mkdir(parents=True, exist_ok=True)
    shutil.copytree(model_dir, f"{path_to_candidates}/{model_id}", dirs_exist_ok=True)
    
    df = data.loc[:, [feat_trgt]]
    df.loc[train.index, 'Group'] = 'Train'
    df.loc[validation.index, 'Group'] = 'Validation'
    df.loc[test.index, 'Group'] = 'Test'
    df['Prediction'] = model.predict(data)
    df['Error'] = df['Prediction'] - df[feat_trgt]
    corrector = LinearBiasCorrector()
    corrector.fit(df.loc[df['Group'] == 'Train', feat_trgt].values, df.loc[df['Group'] == 'Train', 'Prediction'].values)
    df['Prediction Unbiased'] = corrector.predict(df['Prediction'].values)
    df['Error Unbiased'] = df['Prediction Unbiased'] - df[feat_trgt]
    df.to_excel(f"{path_to_candidates}/{model_id}/df.xlsx")
    
    colors_groups = {
        'Train': 'chartreuse',
        'Validation': 'dodgerblue',
        'Test': 'crimson',
    }
    
    df_metrics = pd.DataFrame(
        index=list(colors_groups.keys()),
        columns=[
            'mean_absolute_error', 'pearson_corrcoef', 'bias',
            'mean_absolute_error_unbiased', 'pearson_corrcoef_unbiased', 'bias_unbiased'
        ]
    )
    for group in colors_groups.keys():
        pred = torch.from_numpy(df.loc[df['Group'] == group, 'Prediction'].values)
        pred_unbiased = torch.from_numpy(df.loc[df['Group'] == group, 'Prediction Unbiased'].values)
        real = torch.from_numpy(df.loc[df['Group'] == group, feat_trgt].values)
        df_metrics.at[group, 'mean_absolute_error'] = mean_absolute_error(pred, real).numpy()
        df_metrics.at[group, 'pearson_corrcoef'] = pearson_corrcoef(pred, real).numpy()
        df_metrics.at[group, 'bias'] = np.mean(df.loc[df['Group'] == group, 'Error'].values)
        df_metrics.at[group, 'mean_absolute_error_unbiased'] = mean_absolute_error(pred_unbiased, real).numpy()
        df_metrics.at[group, 'pearson_corrcoef_unbiased'] = pearson_corrcoef(pred_unbiased, real).numpy()
        df_metrics.at[group, 'bias_unbiased'] = np.mean(df.loc[df['Group'] == group, 'Error Unbiased'].values)
    df_metrics.to_excel(f"{path_to_candidates}/{model_id}/metrics.xlsx", index_label="Metrics")
    
    xy_min = df[[feat_trgt, 'Prediction']].min().min()
    xy_max = df[[feat_trgt, 'Prediction']].max().max()
    xy_ptp = xy_max - xy_min
    
    xy_min_unbiased = df[[feat_trgt, 'Prediction Unbiased']].min().min()
    xy_max_unbiased = df[[feat_trgt, 'Prediction Unbiased']].max().max()
    xy_ptp_unbiased = xy_max_unbiased - xy_min_unbiased
    
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(4.5, 4))
    for group in colors_groups.keys():    
        regplot = sns.regplot(
            data=df.loc[df['Group'] == group, :],
            x=feat_trgt,
            y="Prediction",
            label=group,
            color=colors_groups[group],
            scatter_kws=dict(
                linewidth=0.2,
                alpha=0.75,
                edgecolor="k",
                s=20,
            ),
            ax=ax
        )
    bisect = sns.lineplot(
        x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=ax
    )
    ax.set_title(f"{df_sweeps.at[model_id, 'model']} ({df_sweeps.at[model_id, '# Params']} params)")
    ax.set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    ax.set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    plt.gca().set_aspect('equal', adjustable='box')
    fig.savefig(f"{path_to_candidates}/{model_id}/regplot.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/regplot.pdf", bbox_inches='tight')
    plt.close(fig)
    
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(4.5, 4))
    for group in colors_groups.keys():    
        regplot = sns.regplot(
            data=df.loc[df['Group'] == group, :],
            x=feat_trgt,
            y="Prediction Unbiased",
            label=group,
            color=colors_groups[group],
            scatter_kws=dict(
                linewidth=0.2,
                alpha=0.75,
                edgecolor="k",
                s=20,
            ),
            ax=ax
        )
    bisect = sns.lineplot(
        x=[xy_min_unbiased - 0.1 * xy_ptp_unbiased, xy_max_unbiased + 0.1 * xy_ptp_unbiased],
        y=[xy_min_unbiased - 0.1 * xy_ptp_unbiased, xy_max_unbiased + 0.1 * xy_ptp_unbiased],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=ax
    )
    ax.set_title(f"{df_sweeps.at[model_id, 'model']} ({df_sweeps.at[model_id, '# Params']} params)")
    ax.set_xlim(xy_min_unbiased - 0.1 * xy_ptp, xy_max_unbiased + 0.1 * xy_ptp_unbiased)
    ax.set_ylim(xy_min_unbiased - 0.1 * xy_ptp, xy_max_unbiased + 0.1 * xy_ptp_unbiased)
    plt.gca().set_aspect('equal', adjustable='box')
    fig.savefig(f"{path_to_candidates}/{model_id}/regplot_unbiased.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/regplot_unbiased.pdf", bbox_inches='tight')
    plt.close(fig)
    
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(4.5, 4))   
    scatter = sns.scatterplot(
        data=df,
        x=feat_trgt,
        y="Prediction",
        hue="Group",
        palette=colors_groups,
        linewidth=0.2,
        alpha=0.75,
        edgecolor="k",
        s=20,
        hue_order=list(colors_groups.keys()),
        ax=ax
    )
    bisect = sns.lineplot(
        x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=ax
    )
    ax.set_title(f"{df_sweeps.at[model_id, 'model']} ({df_sweeps.at[model_id, '# Params']} params)")
    ax.set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    ax.set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    plt.gca().set_aspect('equal', adjustable='box')
    fig.savefig(f"{path_to_candidates}/{model_id}/scatter.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/scatter.pdf", bbox_inches='tight')
    plt.close(fig)
    
    df_fig = df.loc[:, ['Error', 'Group']]
    groups_rename = {
        group: f"{group}" + "\n" +
               fr"MAE: {df_metrics.at[group, 'mean_absolute_error']:0.2f}" + "\n"
               fr"Pearson $\rho$: {df_metrics.at[group, 'pearson_corrcoef']:0.2f}" + "\n" +
               fr"$\langle$Error$\rangle$: {df_metrics.at[group, 'bias']:0.2f}" 
        for group in colors_groups
    }
    colors_groups_violin = {groups_rename[group]: colors_groups[group] for group in colors_groups}
    df_fig['Group'].replace(groups_rename, inplace=True)
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(7, 4))
    violin = sns.violinplot(
        data=df_fig,
        x='Group',
        y='Error',
        palette=colors_groups_violin,
        scale='width',
        order=list(colors_groups_violin.keys()),
        saturation=0.75,
        legend=False,
        ax=ax
    )
    ax.set_xlabel('')
    fig.savefig(f"{path_to_candidates}/{model_id}/violin.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/violin.pdf", bbox_inches='tight')
    plt.close(fig)
    
    df_fig = df.loc[:, ['Error Unbiased', 'Group']]
    groups_rename = {
        group: f"{group}" + "\n" +
               fr"MAE: {df_metrics.at[group, 'mean_absolute_error_unbiased']:0.2f}" + "\n"
               fr"Pearson $\rho$: {df_metrics.at[group, 'pearson_corrcoef_unbiased']:0.2f}" + "\n" +
               fr"$\langle$Error$\rangle$: {df_metrics.at[group, 'bias_unbiased']:0.2f}" 
        for group in colors_groups
    }
    colors_groups_violin = {groups_rename[group]: colors_groups[group] for group in colors_groups}
    df_fig['Group'].replace(groups_rename, inplace=True)
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(7, 4))
    violin = sns.violinplot(
        data=df_fig,
        x='Group',
        y='Error Unbiased',
        palette=colors_groups_violin,
        scale='width',
        order=list(colors_groups_violin.keys()),
        saturation=0.75,
        legend=False,
        ax=ax
    )
    ax.set_xlabel('')
    fig.savefig(f"{path_to_candidates}/{model_id}/violin_unbiased.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/violin_unbiased.pdf", bbox_inches='tight')
    plt.close(fig)
    
    try:
        explanation = model.explain(data, method=explain_method, baselines=explain_baselines)
        explanation.index = data.index
        explanation.to_excel(f"{path_to_candidates}/{model_id}/explanation.xlsx")
        
        # sns.set_theme(style='whitegrid')
        # fig = shap.summary_plot(
        #     shap_values=explanation.loc[:, feats].values,
        #     features=data.loc[:, feats].values,
        #     feature_names=feats,
        #     max_display=explain_n_feats_to_plot,
        #     plot_type="violin",
        #     show=False,
        # )
        # plt.savefig(f"{path_to_candidates}/{model_id}/explain_beeswarm.png", bbox_inches='tight', dpi=200)
        # plt.savefig(f"{path_to_candidates}/{model_id}/explain_beeswarm.pdf", bbox_inches='tight')
        # plt.close(fig)
        
        sns.set_theme(style='ticks')
        fig = shap.summary_plot(
            shap_values=explanation.loc[:, feats].values,
            features=data.loc[:, feats].values,
            feature_names=feats,
            max_display=explain_n_feats_to_plot,
            plot_type="bar",
            show=False,
            plot_size=[12,8]
        )
        plt.savefig(f"{path_to_candidates}/{model_id}/explain_bar.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_to_candidates}/{model_id}/explain_bar.pdf", bbox_inches='tight')
        plt.close(fig)
    
    except NotImplementedError:
        pass